In [1]:
# read data: 30 ICs 
import nipype.interfaces.io as nio
import os
PD_ICA_file='/templateflow/PD_ICAs';
ICs_list=list(range(30))
ICs_list=["{:0>4d}".format(x) for x in ICs_list]
# file import
ds_ic = nio.DataGrabber(infields=['IC_id'])
ds_ic.inputs.base_directory = PD_ICA_file # database
ds_ic.inputs.template = 'melodic_IC_%4s.nii.gz' # from cwd
ds_ic.inputs.sort_filelist = True
ds_ic.inputs.IC_id = ICs_list
res_ic = ds_ic.run()
ic_list=res_ic.outputs.outfiles
# read data: 3 study groups by sub_list 
GROUPS=['PD','ET','NC']
OUT_DIR='/output/PD_ICA/'
SUB_LIST=[]; AGE_LIST=[]; JCOB_LIST=[];

for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [int(x[0:-1].split('\t')[1]) for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(group_name, ': ', N_sub, sub_list, age_list)
    # grab group Jacobians
    ds_jacobian = nio.DataGrabber(infields=['sub_id'])
    ds_jacobian.inputs.base_directory = current_OUT_DIR # database
    ds_jacobian.inputs.template = '%s_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian.nii.gz' # from cwd
    ds_jacobian.inputs.sort_filelist = True
    ds_jacobian.inputs.sub_id = sub_list
    res_jacobian = ds_jacobian.run()
    jacobian_list=res_jacobian.outputs.outfiles
    JCOB_LIST.append(jacobian_list)
pd_sub_list = SUB_LIST[0]; et_sub_list = SUB_LIST[1]; nc_sub_list = SUB_LIST[2]; 
pd_age_list = AGE_LIST[0]; et_age_list = AGE_LIST[1]; nc_age_list = AGE_LIST[2];
pd_jaco_list=JCOB_LIST[0]; et_jaco_list=JCOB_LIST[1]; nc_jaco_list=JCOB_LIST[2];

PD :  40 ['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0129', 'sub-0132', 'sub-0136', 'sub-1000', 'sub-1020'] [70, 76, 45, 63, 59, 57, 47, 66, 65, 53, 62, 50, 63, 44, 61, 72, 51, 70, 69, 68, 54, 63, 75, 54, 56, 66, 66, 48, 66, 73, 57, 69, 69, 54, 60, 69, 74, 62, 77, 6]
ET :  30 ['sub-0016', 'sub-0061', 'sub-0081', 'sub-0115', 'sub-0119', 'sub-0122', 'sub-0134', 'sub-0178', 'sub-1012', 'sub-1120', 'sub-1160', 'sub-1230', 'sub-1310', 'sub-1340', 'sub-1450', 'sub-1500', 'sub-1690', 'sub-1890', 'sub-1920', 'sub-2400', 'sub-3600', 'sub-3700', 'sub-3900', 'sub-4200', 'sub-4300', 'sub-4700', 'sub-5700', 'sub-7000', 'sub

In [ ]:
# calculate image naive corr
import nibabel as nib
import numpy as np
import time
i_PD_IC=7
PD_ICA_img=nib.load(ic_list[i_PD_IC]);
def cal_groupCorr(ic_img, img_list):
    import nibabel as nib
    from nilearn.image import resample_to_img
    from nilearn.image import math_img
    from scipy import stats
    x_list=[];
    N_sub=len(img_list)
    for i in range(N_sub):
        sub_img=nib.load(img_list[i])
        sub_img_re= resample_to_img(sub_img, ic_img)
        ic_data = ic_img.get_fdata().reshape(-1);
        nz_pos=np.flatnonzero(ic_data)
        ic_val=list(ic_data.ravel()[nz_pos])
        sub_dat = sub_img_re.get_fdata().reshape(-1);
        sub_val=list(stats.zscore(sub_dat.ravel()[nz_pos]))
        x_list.append(np.corrcoef(sub_val, ic_val)[0,1])
    return x_list
t0=time.time()
pd_jaco_corr_list=[cal_groupCorr(nib.load(x), pd_jaco_list) for x in ic_list]; 
et_jaco_corr_list=[cal_groupCorr(nib.load(x), et_jaco_list) for x in ic_list]; 
nc_jaco_corr_list=[cal_groupCorr(nib.load(x), nc_jaco_list) for x in ic_list]; 
print("naive corr of 3 groups take: ", str(time.time()-t0))

In [ ]:
#vis for naive corr
import numpy as np
def plot_corrICA(a, b, c):
    al=np.zeros(len(a)); bl=np.zeros(len(b)); cl=np.zeros(len(c));
    x=list(range(al+bl+cl))
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_title('Group element-wise correlation', color='C0')
    ax.plot(x[0:al-1], a,  label='PD: '+str(round(np.mean(a),4))+'~'+str(round(np.std(a),4)))
    ax.plot(x[al:al+bl-1],b ,  label='NC: '+str(round(np.mean(b),4))+'~'+str(round(np.std(b),4)))
    ax.plot(x[al+bl:al+bl+cl-1], c,  label='ET: '+str(round(np.mean(c),4))+'~'+str(round(np.std(c),4)))
    ax.legend()
plot_corrICA(pd_jaco_corr_list[], nc_jaco_corr_list[], et_jaco_corr_list[])

In [3]:
import statsmodels
import nistats

ModuleNotFoundError: No module named 'nistats'

In [ ]:
from scipy import stats
print('T-test for PD vs NC', stats.ttest_ind(rvs1,rvs2))
print('T-test for PD vs NC', stats.ttest_ind(rvs1,rvs2))

In [ ]:
grou_names=['PD']*len(pd_jaco_x_list_mean)+['ET']*len(et_jaco_x_list_mean)+['NC']*len(nc_jaco_x_list_mean);
age=pd_age_list+et_age_list+nc_age_list;
group_ica_mean=pd_jaco_x_list_mean+et_jaco_x_list_mean+nc_jaco_x_list_mean;
import pandas as pd
df_1=pd.DataFrame({'group':grou_names, 'age':age, 'ica_mean': group_ica_mean})
# generate a boxplot to see the data distribution by genotypes and years. Using boxplot, we can easily detect the 
# differences between different groups
#sns.boxplot(x="group", y="age", hue="ica_mean", data=df_1, palette="Set3") 
#df_1.to_csv('data.csv')
print(df_1)

In [19]:
import pandas as pd
df_1=pd.read_csv('data.csv')
import statsmodels.api as sm
from statsmodels.formula.api import ols
# Ordinary Least Squares (OLS) model
# C(Genotype):C(years) represent interaction term
model= ols('ica_mean ~ (group) + (age) + (group):(age)', data=df_1).fit()
anova_table = sm.stats.anova_lm(model, typ=3)
model1= ols('ica_mean ~ (group) + (age)', data=df_1).fit()
anova_table1 = sm.stats.anova_lm(model1, typ=3)

<KeysViewHDF5 ['HDFVersion', 'ITKVersion', 'OSName', 'OSVersion', 'TransformGroup']>
<KeysViewHDF5 ['TransformType']>


In [ ]:
print(anova_table)
print(model.conf_int())
print(anova_table1)
print(model1.conf_int())

print(model.aic, model.bic)
print(model1.aic, model1.bic)
